In [1]:
%load_ext autoreload

In [2]:
import sys

if '../' not in sys.path:
    sys.path.append('../')

if '../../keyhold' not in sys.path:
    sys.path.append('../../keyhold')

In [3]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta
from fractions import Fraction

In [4]:
%autoreload
from fxtrade.keyhold import KeyHold

from fxtrade.interface.cryptowatch import CryptowatchAPI
from fxtrade.interface.bitflyer import BitflyerAPI

from fxtrade.period import CRangePeriod
from fxtrade.fx import FX

# API キーの読み込み
key_config = KeyHold()

# チャート取得のための API を指定
chart_api = CryptowatchAPI(api_key=key_config['cryptowatch']['x-cw-api-key'])

# 取引のための API を指定
trader_api = BitflyerAPI(api_key=key_config['bitflyer']['api_key'],
                        api_secret=key_config['bitflyer']['api_secret'])

# 取引を統括する FX クラスを定義
fx = FX(name='trader1', origin='JPY', chart_api=chart_api, trader_api=trader_api, data_dir='../data')

# 15分足のチャートを用いて Bitcoin の取引を行うクライアントを定義
fx.generate_client('BTC', crange_period=[CRangePeriod('max', '15m')])

emu = fx.create_emulator(name='emulator1', data_dir='../emulator')

print(fx)

FX(name='trader1',
    origin='JPY',
    data_dir='../data/trader1',
    markets={
        'BTC': Trader(api=BitflyerAPI(api_key='ULkE...', api_secret='IYGp...'),
            code_pair=CodePair(base='BTC', quote='JPY'),
            data_dir='../data/trader1/trader',
            wallet=Wallet({
                'BTC': Stock(BTC, 0.0),
                'JPY': Stock(JPY, 0.0),
            }),
            history=History(n_trades=0),
            chart=Chart(api=CryptowatchAPI(api_key='WLWU...'),
                code_pair=CodePair(base='BTC', quote='JPY'),
                data_dir='../data/trader1/chart',
                crange_period=[CRangePeriod(crange='max', period='15m')],
                board={
                    CRangePeriod(crange='max', period='15m'): Board(name=CRangePeriod(crange='max', period='15m'),
                        api=CryptowatchAPI(api_key='WLWU...'),
                        code_pair=CodePair(base='BTC', quote='JPY'),
                        crange_period=CRangePerio

In [5]:
emu

FX(name='emulator1',
    origin='JPY',
    data_dir='../emulator/emulator1',
    markets={
        'BTC': Trader(api=TraderEmulatorAPI(api=BitflyerAPI, source_dir='../data/trader1/trader'),
            code_pair=CodePair(base='BTC', quote='JPY'),
            data_dir='../emulator/emulator1/trader',
            wallet=Wallet({
                'BTC': Stock(BTC, 0.0),
                'JPY': Stock(JPY, 0.0),
            }),
            history=History(n_trades=0),
            chart=Chart(api=ChartEmulatorAPI(api=CryptowatchAPI, source_dir='../data/trader1/chart'),
                code_pair=CodePair(base='BTC', quote='JPY'),
                data_dir='../emulator/emulator1/chart',
                crange_period=[CRangePeriod(crange='max', period='15m')],
                board={
                    CRangePeriod(crange='max', period='15m'): Board(name=CRangePeriod(crange='max', period='15m'),
                        api=ChartEmulatorAPI(api=CryptowatchAPI, source_dir='../data/trader1/chart'),
 

In [6]:
from fxtrade.stock import Stock

In [7]:
emu.sync_wallet()

| emulator1 | 2023-07-07 00:10:51 +0900 | INFO : FX['BTC'] sync_wallet() <- TraderEmulatorAPI(api=BitflyerAPI, source_dir='../data/trader1/trader') -> ../emulator/emulator1/trader/wallet/wallet.csv. |


{'BTC': Wallet({'JPY': Stock(JPY, 134738.0), 'BTC': Stock(BTC, 5.1e-07)})}

In [8]:
emu['BTC'].wallet['JPY'] = 100000
emu['BTC'].wallet['BTC'] = 0

In [9]:
emu.wallet

Wallet({'JPY': Stock(JPY, 100000.0), 'BTC': Stock(BTC, 0.0)})

In [10]:
from fxtrade.analysis import analyze, estimate_probability, emaverage
from fxtrade.stock import Rate
from fxtrade.trade import Trade

def decide_to_buy(fx, t):
    last_trade = fx['BTC'].get_last_trade()

    if last_trade is None:
        # last trade was SELL
        trade = fx['BTC'].get_max_available(t=t)
        ret = trade % 2
    
    elif last_trade.x.code == 'JPY':
        # last trade was BUY
        best_bid = 1 / fx['BTC'].get_best_bid(t=t)
        
        ret = Trade.from_stock_and_rate(last_trade.x, best_bid).yfloor()
        
        max_available = fx['BTC'].get_max_available(t=t)
        
        if ret.x > max_available.x:
            ret = max_available
        
    else:
        # last trade was SELL
        trade = fx['BTC'].get_max_available(t=t)
        ret = trade % 4
    
    if ret.y < fx['BTC'].minimum_order_quantity():
        return None
    
    return ret

def decide_to_sell(fx, t):
    ret = fx['BTC'].get_max_salable(t=t)
    if ret.x < fx['BTC'].minimum_order_quantity():
        return None
    return ret

def function(fx, t, **kwargs):
    window = 3
    alpha = 0.87
    dt = timedelta(minutes=15)
    
    dt = timedelta(minutes=15) * 500
    charts = fx['BTC'].download_chart(crange_period=['max-15m'], t=(t-dt, t))
    
    chart = charts['max-15m']
    
    df_low = analyze(chart['low'])
    df_high = analyze(chart['high'])

    dif = df_low['diff'].rolling(window).sum()
    prob = estimate_probability(df_low['diff'], window)

    rise = prob[(dif > 0) & (prob < 0.4)]
    fall = prob[(dif < 0) & (prob < 0.01)]

    gmeans = emaverage(df_low['log'], alpha=alpha, dt=dt)
    trend = emaverage(gmeans.diff(), alpha=alpha, dt=dt)

    gm = gmeans.loc[trend.index]
    uptrend = gm.loc[(((trend > 0).astype(float)).diff() > 0).values]
    downtrend = gm.loc[(((trend > 0).astype(float)).diff() < 0).values]
    
    now = df_low.index[-1]
    #print('now:', now)
    
    if len(rise) == 0 or len(fall) == 0:
        is_rising = False
        is_falling = False
        is_after_falling = False
        is_after_rising = False
    else:
        is_rising = rise.index[-1] == now
        is_falling = fall.index[-1] == now

        is_after_falling = fall.index[-1] > rise.index[-1]
        is_after_rising = rise.index[-1] > fall.index[-1]

    if len(uptrend) == 0 or len(downtrend) == 0:
        is_uptrend = False
        is_downtrend = False
    else:
        is_uptrend = uptrend.index[-1] > downtrend.index[-1]
        is_downtrend = uptrend.index[-1] < downtrend.index[-1]

    is_over_gmeans = gmeans.iloc[-1] < df_low['log'].iloc[-1]
    is_under_gmeans = gmeans.iloc[-1] > df_low['log'].iloc[-1]

    ### 買い時
    # アップトレンドかつ平均より下かつ暴落ではない
    # アップトレンドかつ暴騰中（大きく張る）
    # アップトレンドかつ暴落終了直後
    # ダウントレンドかつ暴落終了後の暴騰（大きく張る）

    ### 売り時
    # 暴騰終了直後
    # 暴落開始

    if is_uptrend:
        if is_under_gmeans and (not is_falling):
            print('uptrend: under: not falling: buy some')
            return decide_to_buy(fx, t=now)
        elif is_rising:
            print('uptrend: rinsing: buy more')
            return decide_to_buy(fx, t=now)
        elif is_after_falling:
            print('uptrend: after falling: buy more')
            return decide_to_buy(fx, t=now)
        elif (not is_rising) and is_after_rising:
            print('uptrend: rising end: sell all')
            return decide_to_sell(fx, t=now)
    elif is_downtrend:
        if is_rising:
            print('downtrend: rising: buy more')
            return decide_to_buy(fx, t=now)
        else:
            print('downtrend: sell all')
            return decide_to_sell(fx, t=now)

    if is_falling:
        print('falling: sell all')
        return decide_to_sell(fx, t=now)
    
    return None

In [11]:
hist, wallet_hist = emu.back_test(function, t=(datetime(2023, 5, 14), datetime(2023, 6, 10, 1)), dt=timedelta(minutes=15))

downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: rising: buy more
Wallet({'JPY': Stock(JPY, 100000.0), 'BTC': Stock(BTC, 0.0)})
Trade(2023-07-07 00:10:52.295728 | R(yt/xt): 2.750427504275043e-07 | X(t): 49999.5JPY -> Y(t+dt): 0.013752BTC)
Wallet({'JPY': Stock(JPY, 50022.0), 'BTC': Stock(BTC, 0.01373137)})
downtrend: sell all
Wallet({'JPY': Stock(JPY, 50022.0), 'BTC': Stock(BTC, 0.01373137)})
Trade(2023-07-07 00:10:52.386424 | R(yt/xt): 3640043.763676149 | X(t): 0.01371BTC -> Y(t+dt): 49905.0JPY)
Wallet({'JPY': Stock(JPY, 99964.0), 'BTC': Stock(BTC, 8e-07)})
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sell all
downtrend: sel

In [12]:
hist

{'BTC': History(
     n_trades=372,
     first_timestamp=2023-05-14 11:15:00,
     last_timestamp=2023-06-09 22:00:00
 )}

In [13]:
history = hist['BTC']

In [14]:
history.df

,t,id,from,X(t),to,Y(t+dt),R(yt/xt)
0,2023-05-14 11:15:00,0,JPY,1249449993/25000,BTC,3432843/250000000,1997/7268470000
1,2023-05-14 12:30:00,1,BTC,2746113/200000000,JPY,2497102383/50000,7285492000/2003
2,2023-05-14 22:15:00,2,JPY,24977715539/1000000,BTC,13661477/2000000000,1997/7302358000
3,2023-05-14 22:30:00,3,JPY,2494293841/100000,BTC,6827743/1000000000,1997/7295390000
4,2023-05-14 23:00:00,4,BTC,13658457/1000000000,JPY,3116630769/62500,7312816000/2003
...,...,...,...,...,...,...,...
367,2023-06-09 18:00:00,367,JPY,5366859057/250000,BTC,11540663/2000000000,1997/7429464000
368,2023-06-09 18:15:00,368,JPY,10716880563/500000,BTC,11508711/2000000000,1997/7438404000
369,2023-06-09 18:30:00,369,BTC,17319941/1000000000,JPY,32170030743/500000,7440738000/2003
370,2023-06-09 21:30:00,370,JPY,86136117/4000,BTC,11568621/2000000000,1997/7434500000


In [15]:
history.summarize()

,capital,via,used,earned,position,hold,rate_mean,position_min,hold_min,rate_min,position_max,hold_max,rate_max
0,JPY,BTC,5.971951e+06,5.958313e+06,19099756153/1997000000,661/250000000,1320017/4774939038250,1204389/499250,81/125000000,1997/7434500000,10559686977/1997000000,2961/2000000000,1997/7132514000


In [16]:
wallet_hist

,JPY,BTC,r
2023-05-14 11:15:00,50022.0,1.373137e-02,3.635798e+06
2023-05-14 12:30:00,99964.0,8.000000e-07,3.640044e+06
2023-05-14 22:15:00,74986.0,6.831530e-03,3.653521e+06
2023-05-14 22:30:00,50043.0,1.365927e-02,3.652500e+06
2023-05-14 23:00:00,99909.0,8.100000e-07,3.650902e+06
...,...,...,...
2023-06-09 18:00:00,43346.0,1.156578e-02,3.724208e+06
2023-06-09 18:15:00,21912.0,1.732013e-02,3.724508e+06
2023-06-09 18:30:00,86252.0,1.800000e-07,3.715624e+06
2023-06-09 21:30:00,64717.0,5.784490e-03,3.722647e+06


In [17]:
emu.wallet

Wallet({'JPY': Stock(JPY, 86168.0), 'BTC': Stock(BTC, 8.2e-07)})

In [18]:
df = emu['BTC'].chart.download(t=(datetime(2023, 5, 14), datetime(2023, 6, 10, 1)))['max-15m']
df

,timestamp,open,high,low,close,volume
Date,,,,,,
2023-05-14 00:00:00,1683990000,3642839,3647839,3640226,3640874,3.615247
2023-05-14 00:15:00,1683990900,3642958,3643743,3637042,3642077,7.420800
2023-05-14 00:30:00,1683991800,3643168,3643237,3637753,3641270,4.198293
2023-05-14 00:45:00,1683992700,3641424,3643739,3640491,3642164,4.009302
2023-05-14 01:00:00,1683993600,3642183,3646464,3641372,3645773,3.806255
...,...,...,...,...,...,...
2023-06-10 00:00:00,1686322800,3708320,3713228,3700021,3708743,23.328066
2023-06-10 00:15:00,1686323700,3707386,3711138,3707186,3709127,1.681926
2023-06-10 00:30:00,1686324600,3709249,3711639,3706778,3706923,2.450792


In [19]:
buy = history.df[history.df['from'] == 'JPY']
sell = history.df[history.df['to'] == 'JPY']

In [20]:
buy

,t,id,from,X(t),to,Y(t+dt),R(yt/xt)
0,2023-05-14 11:15:00,0,JPY,1249449993/25000,BTC,3432843/250000000,1997/7268470000
2,2023-05-14 22:15:00,2,JPY,24977715539/1000000,BTC,13661477/2000000000,1997/7302358000
3,2023-05-14 22:30:00,3,JPY,2494293841/100000,BTC,6827743/1000000000,1997/7295390000
5,2023-05-15 00:15:00,5,JPY,2483300949/100000,BTC,13485741/2000000000,1997/7354660000
6,2023-05-15 00:30:00,6,JPY,1548454419/62500,BTC,840737/125000000,1997/7356078000
...,...,...,...,...,...,...,...
364,2023-06-09 14:30:00,364,JPY,10785415979/500000,BTC,11656489/2000000000,1997/7391068000
366,2023-06-09 17:30:00,366,JPY,269034263/12500,BTC,2897647/500000000,1997/7416520000
367,2023-06-09 18:00:00,367,JPY,5366859057/250000,BTC,11540663/2000000000,1997/7429464000
368,2023-06-09 18:15:00,368,JPY,10716880563/500000,BTC,11508711/2000000000,1997/7438404000


In [21]:
df.loc[buy['t']]

,timestamp,open,high,low,close,volume
Date,,,,,,
2023-05-14 11:15:00,1684030500,3633683,3635798,3632543,3635000,3.232990
2023-05-14 22:15:00,1684070100,3651482,3653500,3647157,3649047,7.660207
2023-05-14 22:30:00,1684071000,3649047,3652500,3646156,3651503,2.019163
2023-05-15 00:15:00,1684077300,3665745,3698802,3665102,3678421,51.482231
2023-05-15 00:30:00,1684078200,3678423,3686526,3673285,3678000,15.725512
...,...,...,...,...,...,...
2023-06-09 14:30:00,1686288600,3698003,3700771,3695416,3696467,13.634999
2023-06-09 17:30:00,1686299400,3708243,3718789,3707786,3718030,52.472048
2023-06-09 18:00:00,1686301200,3714065,3724208,3712965,3724208,25.699765


In [22]:
len(history.df)

372

In [23]:
history.df_float

,t,id,from,X(t),to,Y(t+dt),R(yt/xt)
0,2023-05-14 11:15:00,0,JPY,49977.99972,BTC,0.013731,0.0
1,2023-05-14 12:30:00,1,BTC,0.013731,JPY,49942.04766,3637290.064903
2,2023-05-14 22:15:00,2,JPY,24977.715539,BTC,0.006831,0.0
3,2023-05-14 22:30:00,3,JPY,24942.93841,BTC,0.006828,0.0
4,2023-05-14 23:00:00,4,BTC,0.013658,JPY,49866.092304,3650931.602596
...,...,...,...,...,...,...,...
367,2023-06-09 18:00:00,367,JPY,21467.436228,BTC,0.00577,0.0
368,2023-06-09 18:15:00,368,JPY,21433.761126,BTC,0.005754,0.0
369,2023-06-09 18:30:00,369,BTC,0.01732,JPY,64340.061486,3714796.804793
370,2023-06-09 21:30:00,370,JPY,21534.02925,BTC,0.005784,0.0


In [176]:
from fxtrade.history import History, Report

def _close(history, report, df):
    if len(df) == 0:
        return history, report, df
    if len(df) == 1:
        return History.add(Trade.from_series(df.iloc[0])), report, df.drop(df.index[0])
    

def close(self):
    df = self.df.sort_values('t')
    
    head = df.iloc[0]
    tail = df.iloc[1:]
    
    cands = tail[(tail['from'] == head['to']) & (tail['to'] == head['from'])]
    
    trade = Trade.from_series(head)
    
    amount = trade.y
    pop_list = []
    insert_df = None
    for idx, row in cands.iterrows():
        cand = Trade.from_series(row)
        
        if cand.x > amount:
            pair, remain = cand.split_y(amount)
            break
        elif cand.x == amount:
            pair = cand
            remain = None
            break
        else:
            pop_list.append(idx)
            amount -= cand.x
    
    
    display(head)
    display(cands)
    
def split(df, origin: str, return_removed=False):
    removed = df[df['from'] == df['to']]
    
    df_all = df
    df = df.drop(removed.index)
    
    buy = df[df['from'] == origin]
    if len(buy) == 0:
        raise NotImplementedError()
    
    trade_pairs = {}
    for base_name, buy_trades in buy.groupby('to'):
        sell_trades = df[(df['from'] == base_name) & (df['to'] == origin)]
        trade_pairs[base_name] = (buy_trades, sell_trades)
    
    all_idx = set(df_all.index)
    used_idx = set(removed.index)
    for buy_trades, sell_trades in trade_pairs.values():
        used_idx |= set(buy_trades.index)
        used_idx |= set(sell_trades.index)
    
    removed_idx = set(removed.index) | (all_idx - used_idx)
    removed = df_all.loc[sorted(removed_idx)]
    
    return trade_pairs, removed

def settle(buy_df, sell_df):
    buy_df = buy_df.sort_values('R(yt/xt)')
    sell_df = sell_df.sort_values('t')
    
    head = buy_df.iloc[0]
    
    cands = sell_df[sell_df['t'] > head['t']]
    
    display(buy_df['R(yt/xt)'].apply(float))
    display(cands)

In [177]:
_, removed = split(history.df, 'JPY')
removed

,t,id,from,X(t),to,Y(t+dt),R(yt/xt)


In [178]:
#hoge = fx['BTC'].download_history()

In [179]:
trade_pairs, removed = split(hoge.df, 'JPY')

In [180]:
removed

,t,id,from,X(t),to,Y(t+dt),R(yt/xt)
47,2022-04-19 14:29:49.903,209061955,JPY,0,JPY,30000,0
95,2023-05-21 13:40:23.573,268960640,JPY,30000,JPY,0,0
96,2023-05-21 13:40:23.573,268960641,JPY,770,JPY,0,0
97,2023-05-21 13:49:33.530,268961801,JPY,0,JPY,50000,0


In [181]:
for buy_trades, sell_trades in trade_pairs.values():
    settle(buy_trades, sell_trades)

17    1.701979e-07
16    1.702258e-07
19    1.706350e-07
21    1.708519e-07
20    1.709103e-07
22    1.737406e-07
25    1.761759e-07
27    1.764249e-07
28    1.764456e-07
23    1.821630e-07
30    1.825126e-07
29    1.825292e-07
56    1.826145e-07
33    1.829472e-07
32    1.829639e-07
15    1.838846e-07
13    1.842464e-07
36    1.849287e-07
39    1.851115e-07
12    1.857355e-07
35    1.860656e-07
49    1.863619e-07
50    1.864106e-07
54    1.869199e-07
53    1.869269e-07
52    1.869899e-07
57    1.917240e-07
41    1.928185e-07
62    1.956901e-07
58    1.960276e-07
46    1.962475e-07
10    1.964205e-07
43    1.980469e-07
45    1.993812e-07
60    2.018143e-07
0     2.029218e-07
8     2.041163e-07
2     2.045259e-07
5     2.046097e-07
7     2.047076e-07
4     2.047216e-07
67    2.421855e-07
68    2.422508e-07
77    2.469866e-07
70    2.541735e-07
75    2.559457e-07
74    2.559522e-07
78    2.579110e-07
71    2.633841e-07
98    2.652591e-07
88    3.156277e-07
89    3.156386e-07
86    3.1570

,t,id,from,X(t),to,Y(t+dt),R(yt/xt)
18,2022-03-28 21:57:19.743,206746677,BTC,17521/5000000,JPY,20415,102075000000/17521
24,2022-04-03 22:14:43.877,207453316,BTC,22527/2500000,JPY,51080,127700000000/22527
26,2022-04-05 08:14:21.620,207598275,BTC,27533/3125000,JPY,50382,157443750000/27533
31,2022-04-07 00:01:39.580,207798742,BTC,227773/25000000,JPY,49849,1246225000000/227773
34,2022-04-07 01:40:24.350,207808512,BTC,76091/25000000,JPY,16428,410700000000/76091
37,2022-04-08 00:25:03.033,207922181,BTC,7509/2500000,JPY,16115,40287500000/7509
38,2022-04-08 00:27:00.997,207922265,BTC,149429/50000000,JPY,16020,801000000000/149429
40,2022-04-08 09:32:50.080,207943346,BTC,147677/25000000,JPY,31732,793300000000/147677
42,2022-04-11 22:44:03.830,208303699,BTC,7509/2500000,JPY,15429,12857500000/2503
44,2022-04-12 22:48:14.767,208427976,BTC,2503/625000,JPY,20280,12675000000/2503


In [24]:
history.close()

In [25]:
hist.df

,t,id,from,X(t),to,Y(t+dt),R(yt/xt)
0,2023-06-06 04:45:00,275,JPY,10559686977/1997000000,BTC,2961/2000000000,1997/7132514000
1,2023-06-06 22:00:00,298,JPY,168230907/124812500,BTC,47/125000000,1997/7158762000
2,2023-06-08 23:30:00,350,JPY,3688509/12481250,BTC,1/12500000,1997/7377018000
3,2023-06-09 12:30:00,361,JPY,47977241/998500000,BTC,13/1000000000,1997/7381114000
4,2023-06-09 14:30:00,364,JPY,24020971/998500000,BTC,13/2000000000,1997/7391068000
5,2023-06-09 17:30:00,366,JPY,185413/1248125,BTC,1/25000000,1997/7416520000
6,2023-06-09 21:30:00,370,JPY,1204389/499250,BTC,81/125000000,1997/7434500000


In [26]:
hist.df['X(t)'].apply(int)

0    5
1    1
2    0
3    0
4    0
5    0
6    2
Name: X(t), dtype: int64

In [27]:
hist.df['Y(t+dt)'].apply(int)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
Name: Y(t+dt), dtype: int64

In [28]:
(1 / hist.df['R(yt/xt)']).apply(int)

0    3571614
1    3584758
2    3694050
3    3696101
4    3701085
5    3713830
6    3722834
Name: R(yt/xt), dtype: int64

In [29]:
history[0].t

Timestamp('2023-05-14 11:15:00')

In [30]:
ws = wallet_hist['JPY'] + wallet_hist['BTC'] * wallet_hist['r']

In [31]:
hoge = pd.Series(1 - cs, index=df.index).rolling(3).apply(lambda x: x.prod(), raw=True)

NameError: name 'cs' is not defined

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(16, 8))
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()

ax1.plot(df['high'], alpha=0.3, color='C1')
ax1.plot(df['low'], alpha=0.3, color='C2')
ax1.scatter(buy['t'], df.loc[buy['t']]['high'], color='r', s=20)
ax1.scatter(sell['t'], df.loc[sell['t']]['low'], color='b', s=20)
ax1.set_yscale('log')
ax1.grid()

#ax2.plot(ws)
#ax2.plot(df['volume'])
#ax2.plot(df.index, 1-cs, alpha=0.1)
ax2.plot(hoge, alpha=0.1)

q = df['volume'].quantile(0.99)
vol = df[df['volume'] >= q]

ax1.scatter(vol.index, vol['high'].values, color='g', s=50)

#plt.xlim([datetime(2023, 5, 17), datetime(2023, 5, 22)])
plt.xlim([datetime(2023, 5, 25), datetime(2023, 5, 30)])
#plt.xlim([datetime(2023, 6, 5), datetime(2023, 6, 10)])

plt.show()

In [ ]:
plt.hist(df['high'] - df['low'])

In [ ]:
from scipy.stats import gamma

In [ ]:
param = gamma.fit(df['volume'])
model = gamma.freeze(*param)

In [ ]:
cs = model.cdf(df['volume'])

In [ ]:
plt.plot(1 - cs)